<a href="https://colab.research.google.com/github/abdullah75f/function_calling_gemini/blob/main/Final_Water_Tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests google-generativeai pyttsx3 gtts beautifulsoup4 -q
print("Dependencies installed.")

In [ ]:
# --- CELL 2: Imports ---
# (Run this cell after Cell 1)
import os
import requests
import random
import subprocess # Needed for set_volume

from gtts import gTTS
from IPython.display import Audio, display # Ensure display is imported
import google.generativeai as genai
from bs4 import BeautifulSoup
# import pyttsx3 # Keep commented if not used or causes issues in Colab

# Check the GenAI version (optional but good practice)
try:
    print(f"Using google-generativeai version: {genai.__version__}")
except Exception as e:
    print(f"Could not check google-generativeai version: {e}")

print("Libraries imported.")

In [ ]:
# --- CELL 3: API Key Retrieval & Gemini Configuration ---
# (Run this cell after Cell 2. Requires GEMINI_API_KEY secret to be set in Colab)

_in_colab = False
api_key = None
model = None # Initialize model variable

print("Attempting to configure Gemini...")

# Check if running in Colab and import userdata if available
try:
    from google.colab import userdata
    print("Successfully imported google.colab.userdata. Running in Colab.")
    _in_colab = True
except ImportError:
    userdata = None
    print("Warning: 'google.colab.userdata' not found. Assuming not running in Colab.")

# Get the API Key using the appropriate method
try:
    if _in_colab and userdata:
        print("Attempting to retrieve API key from Colab Secrets...")
        # *** Ensure 'GEMINI_API_KEY' matches the name in your Colab Secrets UI ***
        api_key = userdata.get('GEMINI_API_KEY')
        if not api_key:
            raise ValueError("Secret 'GEMINI_API_KEY' not found or access not enabled in Colab Secrets.")
        print("Successfully retrieved API key from Colab Secrets.")

    else: # Fallback for non-Colab environments
        print("Attempting to retrieve API key from environment variables (os.getenv)...")
        # *** Looks for an ENVIRONMENT VARIABLE named 'GEMINI_API_KEY' ***
        api_key = os.getenv('GEMINI_API_KEY')
        if not api_key:
            env_var_source = "environment variables" if not _in_colab else "Colab Secrets (import failed) or environment variables"
            raise ValueError(f"GEMINI_API_KEY not found in {env_var_source}.")
        else:
            print("Successfully retrieved API key from environment variables.")

    # Configure GenAI and Initialize Model (only if key was found)
    if api_key:
        print("Configuring Gemini with the retrieved API key...")
        genai.configure(api_key=api_key)

        print(f"Initializing Gemini model 'gemini-2.0-flash'...")
        # *** Use the correct model name here ***
        model = genai.GenerativeModel('gemini-2.0-flash')
        print(f"Gemini configured and model '{model.model_name}' initialized successfully.")
    else:
        print("ERROR: API Key was not retrieved. Cannot configure Gemini or initialize model.")
        model = None # Ensure model is None if configuration failed

except Exception as e:
    print(f"ERROR: An error occurred during API key retrieval or Gemini configuration: {e}")
    model = None # Ensure model is None if any error occurs

# Final check
if model:
    print("Setup complete. The 'model' variable is ready.")
else:
    print("Setup failed. The 'model' variable is None. Check errors above.")

In [ ]:
# --- CELL 4: Global Variables & Function Definitions ---
# (Run this cell after Cell 3)

print("Defining global variables and functions...")

# --- Global Variables ---
DEFAULT_WATER_TIPS_URL = "https://thewaterproject.org/water_conservation_tips"
WATER_TIPS = [] # Global list to store scraped tips
TIP_FILENAME = "water_conservation_tip.txt"
AUDIO_FILENAME = "tip_generated_audio.mp3"


# --- Function Definitions ---

def scrape_water_tips(url):
    """Scrapes water conservation tips from the given URL."""
    global WATER_TIPS
    print(f"Attempting to scrape tips from: {url}")
    WATER_TIPS = [] # Clear previous tips before scraping
    try:
        response = requests.get(url, timeout=15) # Added timeout
        response.raise_for_status() # Checks for HTTP errors (4xx, 5xx)
        soup = BeautifulSoup(response.content, 'html.parser')

        # --- IMPORTANT: Inspect the target website's HTML to find the correct selector ---
        # Example: Find list items within a specific div
        # container = soup.find('div', class_='content-area') # Adjust selector
        # if container:
        #    tip_elements = container.find_all('li')
        # else:
        #    tip_elements = soup.find_all('li') # Fallback to broad selector
        tip_elements = soup.find_all('li') # Using the original broad selector - refine if needed
        # ------------------------------------------------------------------------------

        if not tip_elements:
             print(f"Warning: No 'li' elements found on {url} using the current selector. Scraping might have failed or the selector needs adjustment.")
             return # Exit function early if no elements found

        scraped_list = [tip.get_text(strip=True) for tip in tip_elements if tip.get_text(strip=True)] # Use get_text() and filter empty strings

        if not scraped_list:
            print(f"Warning: Found 'li' elements but they contained no usable text after stripping whitespace.")
        else:
            WATER_TIPS = scraped_list # Assign to global variable ONLY if successful
            print(f"Scraped {len(WATER_TIPS)} potential water conservation tips.")
            # print("Sample tips:", WATER_TIPS[:5]) # Optional: print first few tips

    except requests.exceptions.Timeout:
        print(f"Error: Request timed out while fetching URL: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
    except Exception as e:
        # Catch potential errors during parsing
        print(f"Error scraping or parsing tips from {url}: {e}")


def get_water_conservation_tip(url: str = DEFAULT_WATER_TIPS_URL):
    """
    Returns a random water conservation tip. Scrapes if needed.
    Uses the provided URL, defaulting to DEFAULT_WATER_TIPS_URL.
    """
    global WATER_TIPS
    needs_scrape = False
    # Basic check: Scrape if list is empty or if a non-default URL is requested
    # (Assumes tips are from default URL unless explicitly asked for another)
    if not WATER_TIPS:
        print("Global tip list (WATER_TIPS) is empty. Triggering scrape.")
        needs_scrape = True
    elif url != DEFAULT_WATER_TIPS_URL:
         # If you want to scrape *every time* a non-default URL is given, use this:
         print(f"Requested URL '{url}' differs from default. Triggering fresh scrape.")
         needs_scrape = True
         # If you only want to scrape non-default URLs if the *current* tips aren't from that URL,
         # you'd need to store the last scraped URL alongside WATER_TIPS.

    if needs_scrape:
        scrape_water_tips(url) # Scrape new results using the requested URL

    if WATER_TIPS:
        selected_tip = random.choice(WATER_TIPS)
        print(f"Returning random tip: '{selected_tip}'")
        return selected_tip
    else:
        # This message is returned if scraping failed or yielded no tips
        print("No water conservation tips available in WATER_TIPS after checking/scraping.")
        return "Sorry, no water conservation tips could be retrieved at this time."


def save_tip(tip: str):
    """Saves the provided water conservation tip to a text file."""
    global TIP_FILENAME
    if not tip or tip.startswith("Sorry, no water"): # Avoid saving error messages or empty strings
        print("Invalid or empty tip provided. Not saving.")
        return "No valid tip provided to save."

    try:
        with open(TIP_FILENAME, "w", encoding='utf-8') as file: # Added encoding
            file.write(tip)
        print(f"Tip successfully saved to {TIP_FILENAME}")
        return f"Tip saved to {TIP_FILENAME}"
    except Exception as e:
        print(f"Error saving tip to file '{TIP_FILENAME}': {e}")
        return f"Error saving tip: {e}"


def set_volume(level: int):
    """Sets the system volume using pactl (Linux specific)."""
    # Input validation
    if not isinstance(level, int) or not 0 <= level <= 150: # Allow up to 150%? Check pactl capabilities
         print("Error: Volume level must be an integer between 0 and 150 (or 100 depending on system).")
         return "Error: Volume level must be an integer between 0 and 150." # Adjust range if needed

    # Check OS - pactl is typically Linux
    if os.name != 'posix':
        print("Info: Volume control via 'pactl' is Linux-only. This command might not work.")
        # Decide whether to return an error or just a warning
        return "Warning: 'pactl' might not work on this OS."

    try:
        # Use check_call for better error reporting if pactl fails
        command = ["pactl", "set-sink-volume", "@DEFAULT_SINK@", f"{level}%"]
        print(f"Executing command: {' '.join(command)}")
        # Using run instead of check_call to capture output/errors if needed
        result = subprocess.run(command, capture_output=True, text=True, check=False, timeout=5) # check=False, add timeout

        if result.returncode == 0:
            print(f"pactl command executed successfully (Return Code: {result.returncode}).")
            # pactl success doesn't always mean volume *changed* noticeably, but the command ran.
            return f"Attempted to set volume to {level}% via pactl."
        else:
            # Log pactl errors if any
            error_message = result.stderr.strip() if result.stderr else result.stdout.strip()
            # Sometimes error messages go to stdout
            if not error_message: error_message = "Unknown pactl error"
            print(f"Error executing pactl command (Return Code: {result.returncode}): {error_message}")
            return f"Error setting volume via pactl: {error_message}"

    except FileNotFoundError:
        print("Error: 'pactl' command not found. Is pulse audio installed and in PATH?")
        return "Error: 'pactl' command not found."
    except subprocess.TimeoutExpired:
        print("Error: 'pactl' command timed out.")
        return "Error: pactl command timed out."
    except Exception as e:
        # Catch other potential errors
        print(f"An unexpected error occurred setting volume: {e}")
        return f"An unexpected error occurred setting volume: {e}"


def tell_tip():
    """Reads the water conservation tip from the file aloud using gTTS."""
    global TIP_FILENAME, AUDIO_FILENAME

    try:
        # Check if the tip file exists
        if not os.path.exists(TIP_FILENAME):
            print(f"Error: Tip file '{TIP_FILENAME}' not found. Please save a tip first.")
            return f"Error: Tip file '{TIP_FILENAME}' not found."

        with open(TIP_FILENAME, "r", encoding='utf-8') as file: # Added encoding
            tip = file.read().strip() # Read and remove leading/trailing whitespace

        if not tip:
             print(f"Error: Tip file '{TIP_FILENAME}' is empty.")
             return f"Error: Tip file '{TIP_FILENAME}' is empty."

        # Clean up previous audio file if it exists
        if os.path.exists(AUDIO_FILENAME):
            try:
                os.remove(AUDIO_FILENAME)
                print(f"Removed existing audio file: {AUDIO_FILENAME}")
            except OSError as e:
                print(f"Warning: Could not remove existing audio file {AUDIO_FILENAME}: {e}")


        print(f"Generating audio for tip: '{tip}'")
        # Use gTTS to create the audio file
        tts = gTTS(text=f"Here is a water conservation tip: {tip}", lang='en', slow=False) # slow=False for normal speed
        tts.save(AUDIO_FILENAME)

        print(f"Audio saved to {AUDIO_FILENAME}. Attempting to play...")
        # Display and play the audio in Colab/IPython
        display(Audio(AUDIO_FILENAME, autoplay=True))

        # Note: Audio playback might take a moment to start in Colab.
        # The os.remove call might happen before playback finishes if uncommented below.

        # Optional: Clean up the audio file *after* some delay or keep it
        # try:
        #     # import time; time.sleep(5) # Example: wait 5 seconds
        #     os.remove(AUDIO_FILENAME)
        #     print(f"Cleaned up audio file: {AUDIO_FILENAME}")
        # except OSError as e:
        #     print(f"Warning: Could not remove audio file {AUDIO_FILENAME}: {e}")

        return "Tip audio generated and playback initiated."

    except Exception as e:
        print(f"Error during text-to-speech process: {e}")
        return f"Error telling tip: {e}"

print("Global variables and functions defined.")

In [ ]:
# --- CELL 5: Usage Examples ---
# (Run this cell after Cell 4. Assumes Cell 3 ran successfully and 'model' is not None)

print("\n--- Starting Usage Examples ---")

# Check if the model is available before proceeding
if not model:
    print("ERROR: Gemini model ('model' variable) is not initialized. Cannot run examples. Please check Cell 3 for errors.")
else:
    # --- Example 1: Generate a unique tip using the LLM ---
    print("\n--- Example 1: Generating a unique tip ---")
    prompt_llm = "Provide an original water conservation tip suitable for a household kitchen. The tip should be creative and not commonly found on standard lists. Do not cite any source. Keep it concise (1-2 sentences)."
    try:
        print(f"Sending prompt to Gemini: '{prompt_llm}'")
        response = model.generate_content(prompt_llm)
        # Accessing the text part safely
        if response.parts:
             generated_tip = response.text # .text usually works for simple text responses
        else:
             # Handle cases where response might be blocked or empty
             print(f"Warning: Received an empty or blocked response from Gemini. Prompt: '{prompt_llm}', Finish Reason: {response.prompt_feedback}")
             generated_tip = None

        if generated_tip:
            print("\n--- LLM Generated Tip ---")
            print(generated_tip)
            print("-------------------------\n")

            # Example 1b: Save and speak the generated tip
            save_status = save_tip(generated_tip)
            print(f"Save status: {save_status}")
            if TIP_FILENAME in save_status: # Check if saving seemed successful
                speak_status = tell_tip()
                print(f"Speak status: {speak_status}")
        else:
            print("Could not generate a tip with the LLM.")

    except Exception as e:
        print(f"ERROR: Error generating content with Gemini: {e}")


    # --- Example 2: Get a tip from the web, save it, and speak it ---
    print("\n--- Example 2: Getting a tip from the web ---")
    # Using the default URL defined earlier
    web_tip = get_water_conservation_tip() # Fetches from DEFAULT_WATER_TIPS_URL

    if web_tip and not web_tip.startswith("Sorry, no water"):
        print(f"\nRetrieved web tip: '{web_tip}'")
        save_status = save_tip(web_tip)
        print(f"Save status: {save_status}")
        if TIP_FILENAME in save_status: # Check if saving seemed successful
            speak_status = tell_tip()
            print(f"Speak status: {speak_status}")
    else:
        print("Could not retrieve a valid tip from the web using get_water_conservation_tip().")
        print(f"(Attempted URL: {DEFAULT_WATER_TIPS_URL})")


    # --- Example 3: Set volume (if on Linux and pactl is available) ---
    print("\n--- Example 3: Setting Volume (Linux/pactl only) ---")
    volume_status = set_volume(75) # Attempt to set volume to 75%
    print(f"Volume set status: {volume_status}")

# --- End of Examples ---
print("\n--- Script Finished ---")